In [7]:
import asyncio
import pandas as pd
import plotly.graph_objs as go
import plotly.express as px

import warnings
warnings.filterwarnings("ignore")


#df_c4z = pd.read_csv("order_test_fet_15_11_c4z.csv")
df_26nf = pd.read_csv("csv/test.csv")
#data_c4z=data_c4z[:1000]
#data_26nf=data_26nf[:1000]

#df_index_price = pd.read_csv("2024-11-14.csv")
#df_index_price=df_index_price[:1000]


In [8]:
# #df_c4z.drop(columns=['marketId', 'subaccountId', 'executionType', 'orderHash','quantity'], inplace=True)
df_26nf.drop(columns=['marketId', 'subaccountId', 'executionType', 'orderHash'], inplace=True)


#df_26nf['price'] = df_26nf['price'] * 10**12
#df_c4z['price'] = df_c4z['price'] * 10**12

In [9]:
df_26nf

,orderType,price,quantity,updated_at,state,created_at
0,buy,36640000,1380.55,2025-01-23 09:05:44.009,canceled,2025-01-23 09:05:42.862
1,buy,36610000,187.46,2025-01-23 09:05:44.009,canceled,2025-01-23 09:05:42.862
2,sell,36800000,187.46,2025-01-23 09:05:44.009,canceled,2025-01-23 09:05:42.862
3,buy,36650000,1380.55,2025-01-23 09:05:44.009,booked,2025-01-23 09:05:44.009
4,buy,36620000,187.40,2025-01-23 09:05:44.009,booked,2025-01-23 09:05:44.009
...,...,...,...,...,...,...
22691,buy,37020000,1727.57,2025-01-23 05:05:49.118,canceled,2025-01-23 05:05:46.874
22692,sell,37150000,0.14,2025-01-23 05:05:49.118,canceled,2025-01-23 05:05:46.874
22693,buy,37020000,1727.57,2025-01-23 05:05:46.874,canceled,2025-01-23 05:05:46.183
22694,buy,37020000,0.14,2025-01-23 05:05:46.874,canceled,2025-01-23 05:05:46.183


In [10]:
df_transformed = df_26nf.pivot_table(
    index=["updated_at", "state", "created_at"],
    columns="orderType",
    values="price",
    aggfunc="first"
).reset_index()

print(df_transformed.columns) 
# Rename columns for clarity
#df_transformed.columns = ["updated_at", "state", "created_at", "buy_price", "sell_price"]
df_transformed["buy"]= df_transformed["buy"]/1e6
df_transformed["sell"]= df_transformed["sell"]/1e6
#df_transformed["buy_po"]= df_transformed["buy_po"]/1e6
#df_transformed["sell_po"]= df_transformed["sell_po"]/1e6
# Display the transformed DataFrame
df_transformed["offset"] = 100 * (df_transformed["sell"]-df_transformed["buy"])/ ((df_transformed["sell"]+df_transformed["buy"])/2)/2
df_transformed

Index(['updated_at', 'state', 'created_at', 'buy', 'sell'], dtype='object', name='orderType')


orderType,updated_at,state,created_at,buy,sell,offset
0,2025-01-23 05:05:46.874,canceled,2025-01-23 05:05:46.183,37.02,37.14,0.161812
1,2025-01-23 05:05:49.118,canceled,2025-01-23 05:05:46.874,37.02,37.15,0.175273
2,2025-01-23 05:05:51.091,canceled,2025-01-23 05:05:49.118,37.04,37.16,0.161725
3,2025-01-23 05:05:55.340,canceled,2025-01-23 05:05:51.091,37.03,37.15,0.161769
4,2025-01-23 05:05:57.147,canceled,2025-01-23 05:05:55.340,37.04,37.16,0.161725
...,...,...,...,...,...,...
7544,2025-01-23 09:05:39.723,canceled,2025-01-23 09:05:37.796,36.65,36.81,0.217806
7545,2025-01-23 09:05:41.451,canceled,2025-01-23 09:05:39.723,36.67,36.82,0.204109
7546,2025-01-23 09:05:42.862,canceled,2025-01-23 09:05:41.451,36.66,36.81,0.204165
7547,2025-01-23 09:05:44.009,booked,2025-01-23 09:05:44.009,36.65,36.81,0.217806


In [11]:
import plotly.graph_objects as go


fig = go.Figure()

# Add traces for each y-column
fig.add_trace(go.Scatter(x=df_transformed['updated_at'], y=df_transformed['buy'],
                         mode='lines', name='buy '))
fig.add_trace(go.Scatter(x=df_transformed['updated_at'], y=df_transformed['sell'],
                         mode='lines', name='sell '))
#fig.add_trace(go.Scatter(x=df_transformed['updated_at'], y=df_transformed['buy_po'],
#                         mode='lines', name='buy '))
#fig.add_trace(go.Scatter(x=df_transformed['updated_at'], y=df_transformed['sell_po'],
#                         mode='lines', name='sell '))
# Update layout for titles and axes
fig.update_layout(
    title="Offsets over Time",
    xaxis_title="Updated At",
    yaxis_title="Offset Value",
    legend_title="Offsets",
)

# Show the figure
fig.show()


In [12]:
import plotly.express as px

# Create a sample DataFrame

# Create a line plot from the DataFrame
fig = px.line(df_transformed, x='updated_at', y='offset', title='Offset from c4z order')

# Show the plot
fig.show()
